In [0]:
from pyspark.sql.functions import col, when, lit
df_category = spark.read.table("imdb_dev.silver.Category") 
df_episode = spark.read.table("imdb_dev.silver.TitleEpisode")
df_genre = spark.read.table("imdb_dev.silver.Genre") 
df_genre_of_title = spark.read.table("imdb_dev.silver.GenreOfTitle") 
df_known_for_participation = spark.read.table("imdb_dev.silver.KnownForParticipation")
df_participation = spark.read.table("imdb_dev.silver.Participation") 
df_person = spark.read.table("imdb_dev.silver.Person") 
df_primary_professions = spark.read.table("imdb_dev.silver.PersonPrimaryProfession") 
df_professions = spark.read.table("imdb_dev.silver.Profession")  
df_rating = spark.read.table("imdb_dev.silver.TitleRating") 
df_region = spark.read.table("imdb_dev.silver.Region") 
df_title = spark.read.table("imdb_dev.silver.Title") 
df_title_AKAS = spark.read.table("imdb_dev.silver.TitleAKAS") 
df_title_type = spark.read.table("imdb_dev.silver.TitleType") 
df_type_AKAS = spark.read.table("imdb_dev.silver.TypeAKAS") 
df_type_of_alternative = spark.read.table("imdb_dev.silver.TypeOfAlternative") 

In [0]:
df_participation = df_participation.join(
    df_known_for_participation.withColumnRenamed("personId", "kfp_personId").withColumnRenamed("titleId", "kfp_titleId"),
    (df_participation["personId"] == col("kfp_personId")) & (df_participation["titleId"] == col("kfp_titleId")),
    "left"
)
df_participation = df_participation.withColumn(
    "isKnownForParticipation",
    when(col("kfp_personId").isNotNull(), lit(True)).otherwise(lit(False))
).drop("kfp_personId", "kfp_titleId", "characterName")

In [0]:
df_genre_of_title = df_genre_of_title.join(df_genre, on="genreId", how="left")

df_type_of_alternative = df_type_of_alternative.join(df_type_AKAS, on="typeAKASId", how="left")

df_primary_professions = df_primary_professions.join(df_professions, on="professionId", how="left")

In [0]:
df_obt = (
    df_participation
    .join(df_title, on="titleId", how="left").drop("originalTitle")
    .join(df_title_type, on="titleTypeId", how="left")
    .join(df_rating, on="titleId", how="left")
    .join(df_genre_of_title, on="titleId", how="left")
    .join(df_category, on="categoryId", how="left")
    .join(df_person, on="personId", how="left")
    .join(df_primary_professions, on="personId", how="left")
    .join(df_episode, on="titleId", how="left")
    .join(df_title_AKAS, on="titleId", how="left").drop("title")
    .join(df_region, on="regionId", how="left")
    .join(df_type_of_alternative, on="titleAKASId", how="left")
)

In [0]:
df_obt = df_obt.select("participationId", "titleId", "titleTypeId", "genreId", "titleAKASId", "typeAKASId", "personId", "categoryId", "professionId", "parentTitleId", "regionId", 
                       "primaryTitle", "isAdult", "startYear", "endYear", "runtimeMinutes", "genre", "averageRating", "numVotes", "titleType", "seasonNumber", 
                       "episodeNumber", "region", "typeAKAS", "category", "isKnownForParticipation", "personName", "birthYear", "deathYear", "profession")
df_obt.write.format("delta").mode("overwrite").insertInto("imdb_dev.gold.OneBigTable")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can